## **Notebook 04: Gradio Web Interface**

### Purpose
Build a polished, user-facing web interface for Radar using Gradio. This transforms our agent pipeline into an accessible demo that can be deployed and shared.

### What We'll Do

| Step | Task | Output |
|------|------|--------|
| 1 | **Setup Gradio** | Import libraries and configure interface |
| 2 | **Design Layout** | Create card-based UI with sections |
| 3 | **Build Processing Function** | Connect Gradio to agent pipeline |
| 4 | **Add ArXiv Input** | Allow users to input paper IDs |
| 5 | **Style Interface** | Custom CSS for professional appearance |
| 6 | **Test Locally** | Run and validate interface |
| 7 | **Deployment Prep** | Package for Hugging Face Spaces |

### Key Questions to Answer
- How do we structure the output for maximum readability?
- What input methods work best (ArXiv ID vs file upload)?
- How do we handle loading states and errors gracefully?
- What styling makes this look portfolio-quality?

### Expected Outcomes
- Working Gradio interface running locally
- Clean, card-based layout for results
- Professional styling with gradients and spacing
- Ready-to-deploy application file

### Design Philosophy
Focus on clarity and scannability. Users should immediately understand:
1. What the paper is about (Two-line summary)
2. What problem it solves (Challenge)
3. How it solves it (Solution)
4. Technical details (Key points)

---



In [2]:
# Cell 2: Imports and Setup

"""
Import Gradio and connect to our existing agent pipeline.
"""

import gradio as gr
import json
import os
from datetime import datetime

# LangGraph and agents
from langgraph.graph import StateGraph, END
from langchain_anthropic import ChatAnthropic
from typing import TypedDict

# ArXiv
import arxiv

# PDF processing
import fitz

# Environment
from dotenv import load_dotenv
load_dotenv()

# Verify API key
api_key = os.getenv('ANTHROPIC_API_KEY')
if not api_key:
    raise ValueError("ANTHROPIC_API_KEY not found")

# Initialize Claude client
llm = ChatAnthropic(
    model="claude-sonnet-4-20250514",
    temperature=0.7,
    max_tokens=4096
)

print("Setup complete")
print(f"Gradio version: {gr.__version__}")
print("Ready to build interface")

Setup complete
Gradio version: 6.3.0
Ready to build interface


**Starting Out**

- I will plug in the same agent schema I used in the Agent Prototype process 

- Starting by plugging in the Paper Analyzer Agent 

- Then following up with the Simplifier Agent 

In [3]:
# Cell 3: Load Agent Pipeline from Notebook 03

"""
Import the agent state and functions we built in Notebook 03.
"""

# Define state schema (same as Notebook 03)
class AgentState(TypedDict):
    paper_title: str
    paper_text: str
    paper_sections: dict
    technical_summary: str
    key_methods: str
    main_results: str
    limitations: str
    executive_summary: str
    key_innovation: str
    accessible_explanation: str
    technical_points: str
    processing_stage: str
    errors: list

# Paper Analyzer Agent
def paper_analyzer_agent(state: AgentState) -> AgentState:
    prompt = f"""You are an expert AI researcher analyzing academic papers. 

Paper Title: {state['paper_title']}

Paper Content (excerpt):
{state['paper_text'][:8000]}

Your task: Extract the following technical insights:

1. TECHNICAL SUMMARY (2-3 sentences): What problem does this solve and how?
2. KEY METHODS (bullet points): What techniques/approaches were used?
3. MAIN RESULTS (bullet points): What were the key findings or performance metrics?
4. LIMITATIONS (bullet points): What are the acknowledged limitations or future work needed?

Respond ONLY with valid JSON, no markdown formatting:
{{
  "technical_summary": "...",
  "key_methods": ["...", "..."],
  "main_results": ["...", "..."],
  "limitations": ["...", "..."]
}}"""

    try:
        response = llm.invoke(prompt)
        content = response.content.strip()
        
        if content.startswith('```'):
            content = content.split('```')[1]
            if content.startswith('json'):
                content = content[4:]
        content = content.strip()
        
        result = json.loads(content)
        
        state['technical_summary'] = result['technical_summary']
        state['key_methods'] = '\n'.join(f"- {m}" for m in result['key_methods'])
        state['main_results'] = '\n'.join(f"- {r}" for r in result['main_results'])
        state['limitations'] = '\n'.join(f"- {l}" for l in result['limitations'])
        state['processing_stage'] = 'analyzed'
        
    except Exception as e:
        state['errors'].append(f"Analyzer error: {str(e)}")
        state['processing_stage'] = 'analyzer_failed'
    
    return state

# Simplifier Agent
def simplifier_agent(state: AgentState) -> AgentState:
    prompt = f"""You are an expert science communicator making AI research accessible.

Paper Title: {state['paper_title']}
Technical Summary: {state['technical_summary']}
Key Methods: {state['key_methods']}
Main Results: {state['main_results']}

Create a clear explanation using this structure:

1. TWO-LINE SUMMARY: What is this and why does it matter? Maximum 2 sentences.
2. THE CHALLENGE (3-4 bullet points): What problem exists? What are researchers trying to solve?
3. WHAT THIS PAPER DOES (1 paragraph): Explain the approach in simple terms.
4. KEY TECHNICAL POINTS (3-5 bullet points): Break down technical aspects into simple language.

Respond ONLY with valid JSON, no markdown formatting:
{{
  "two_line_summary": "...",
  "challenge_bullets": ["...", "...", "..."],
  "solution_overview": "...",
  "technical_points": ["...", "...", "..."]
}}"""

    try:
        response = llm.invoke(prompt)
        content = response.content.strip()
        
        if content.startswith('```'):
            content = content.split('```')[1]
            if content.startswith('json'):
                content = content[4:]
        content = content.strip()
        
        result = json.loads(content)
        
        state['executive_summary'] = result['two_line_summary']
        state['key_innovation'] = '\n'.join(f"- {c}" for c in result['challenge_bullets'])
        state['accessible_explanation'] = result['solution_overview']
        state['technical_points'] = '\n'.join(f"- {t}" for t in result['technical_points'])
        state['processing_stage'] = 'simplified'
        
    except Exception as e:
        state['errors'].append(f"Simplifier error: {str(e)}")
        state['processing_stage'] = 'simplifier_failed'
    
    return state

# Build workflow
workflow = StateGraph(AgentState)
workflow.add_node("analyzer", paper_analyzer_agent)
workflow.add_node("simplifier", simplifier_agent)
workflow.set_entry_point("analyzer")
workflow.add_edge("analyzer", "simplifier")
workflow.add_edge("simplifier", END)
app = workflow.compile()

print("Agent pipeline loaded and compiled")
print("Ready to process papers through Gradio")

Agent pipeline loaded and compiled
Ready to process papers through Gradio


We want to define a simple workflow for Gradio: 
- Look up a paper's ArXiv ID
- Put the paper through some defined processing 
- Give the back the results in the precise format we defined 


In [4]:
# Core Processing Function for Gradio

"""
This function connects Gradio inputs to our agent pipeline.
Takes an ArXiv ID, processes the paper, returns formatted results.
"""

def process_arxiv_paper(arxiv_id):
    """
    Main function that Gradio will call.
    
    Args:
        arxiv_id (str): ArXiv paper ID (e.g., "2601.05245")
    
    Returns:
        tuple: (summary, challenge, solution, technical_points, error_message)
    """
    
    try:
        # Step 1: Download paper from ArXiv
        search = arxiv.Search(id_list=[arxiv_id])
        client = arxiv.Client()
        paper = next(client.results(search))
        
        # Step 2: Download and extract PDF text
        pdf_path = f"temp_{arxiv_id}.pdf"
        paper.download_pdf(filename=pdf_path)
        
        # Extract text
        doc = fitz.open(pdf_path)
        full_text = ""
        for page_num in range(min(len(doc), 20)):  # Limit to first 20 pages
            full_text += doc[page_num].get_text()
        doc.close()
        
        # Clean up PDF
        os.remove(pdf_path)
        
        # Step 3: Prepare state for agents
        initial_state = {
            'paper_title': paper.title,
            'paper_text': full_text,
            'paper_sections': {},
            'technical_summary': '',
            'key_methods': '',
            'main_results': '',
            'limitations': '',
            'executive_summary': '',
            'key_innovation': '',
            'accessible_explanation': '',
            'technical_points': '',
            'processing_stage': 'initialized',
            'errors': []
        }
        
        # Step 4: Run through agent pipeline
        result = app.invoke(initial_state)
        
        # Step 5: Check for errors
        if result['processing_stage'] != 'simplified':
            error_msg = f"Processing failed: {', '.join(result['errors'])}"
            return "", "", "", "", error_msg
        
        # Step 6: Return formatted results
        return (
            result['executive_summary'],          # Summary
            result['key_innovation'],             # Challenge  
            result['accessible_explanation'],     # Solution
            result['technical_points'],           # Technical points
            ""                                    # No error
        )
        
    except StopIteration:
        return "", "", "", "", f"Paper not found: {arxiv_id}"
    except Exception as e:
        return "", "", "", "", f"Error: {str(e)}"

print("Processing function defined: process_arxiv_paper()")
print("Takes ArXiv ID -> Returns formatted results")

Processing function defined: process_arxiv_paper()
Takes ArXiv ID -> Returns formatted results


In [5]:
# Cell 5: Create Gradio Interface

"""
Build the user-facing web interface using Gradio Blocks.
Layout: Input section -> Output cards with results
"""

with gr.Blocks(title="Radar - AI Research Intelligence") as demo:
    
    # Header
    gr.Markdown("""
    # 🔭 Radar - AI Research Intelligence
    
    Automatically analyze and translate AI research papers from ArXiv into accessible insights.
    """)
    
    # Input section
    with gr.Row():
        with gr.Column(scale=3):
            arxiv_input = gr.Textbox(
                label="ArXiv Paper ID",
                placeholder="Enter ArXiv ID (e.g., 2601.05245)",
                info="Find paper IDs on arxiv.org"
            )
        with gr.Column(scale=1):
            submit_btn = gr.Button("Analyze Paper", variant="primary", size="lg")
    
    # Error/status message
    error_msg = gr.Textbox(label="Status", visible=False)
    
    # Output section
    gr.Markdown("## 📊 Analysis Results")
    
    with gr.Row():
        with gr.Column():
            summary_output = gr.Textbox(
                label="Two-Line Summary",
                lines=3,
                interactive=False
            )
    
    with gr.Row():
        with gr.Column():
            challenge_output = gr.Textbox(
                label="The Challenge",
                lines=6,
                interactive=False
            )
    
    with gr.Row():
        with gr.Column():
            solution_output = gr.Textbox(
                label="What This Paper Does",
                lines=5,
                interactive=False
            )
    
    with gr.Row():
        with gr.Column():
            technical_output = gr.Textbox(
                label="Key Technical Points",
                lines=6,
                interactive=False
            )
    
    # Connect button to processing function
    submit_btn.click(
        fn=process_arxiv_paper,
        inputs=arxiv_input,
        outputs=[summary_output, challenge_output, solution_output, technical_output, error_msg]
    )
    
    # Example papers
    gr.Markdown("""
    ### Example Papers to Try:
    - `2601.05245` - Manifold limit for graph neural networks
    - `2601.06022` - AdaFuse ensemble decoding for LLMs
    """)

print("Gradio interface built")
print("Ready to launch")

Gradio interface built
Ready to launch


In [ ]:
# Cell 6: Launch Gradio Interface

"""
Launch the interface locally.
Opens in browser at http://127.0.0.1:7860
"""

demo.launch(
    share=False,        # Set to True to get public URL for sharing
    server_port=7860,   # Port number
    show_error=True     # Show detailed errors in interface
)

print("Interface launched")
print("Open browser to: http://127.0.0.1:7860")
print("Press Ctrl+C to stop server")